In [1]:
import pandas as pd
import numpy as np
import os
import sys

_data_dir = 'data'
_preprocess_dir = 'data\\preprocess'
_predict_dir = 'data\\predict'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

train_df = pd.read_csv(os.path.join(_preprocess_dir, 'train_features.csv'))

train_df.label = train_df.label.map({2: 1, 1: 1, 0: 0})

X = train_df.drop('label', axis=1)
y = train_df.label

print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

zero_count = y.tolist().count(0)
one_count = y.tolist().count(1)
two_count = y.tolist().count(2)
print('All:  0: %f; 1: %f ; 2: %f' % (zero_count * 1.0 / len(y), one_count * 1.0 / len(y), two_count * 1.0 / len(y)))

X.shape:  (1269117, 38)
y.shape:  (1269117,)
All:  0: 0.937196; 1: 0.062804 ; 2: 0.000000


In [2]:
# DNN
MODEL_PATH="model"

from keras.layers import Input, BatchNormalization, concatenate, Flatten, Dropout, Dense,ReLU
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard

C:\Users\IfyourRtheone\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
print(y.shape)
print(X.shape)
X.head()

(1269117,)
(1269117, 38)


,user_id,id,gender,install_city_match,cargo_type,handling_type,truck_count,highway,lcl_cargo,truck_type_cargo_match_first,truck_type_cargo_match_second,truck_type_cargo_match_third,truck_type_cargo_match_fourth,truck_type_driver_match_first,truck_len_match,regular_subscribe_match,regular_search_3_match,regular_search_7_match,regular_search_14_match,regular_search_30_match,line_match,line_30days_match,line_60days_match,scan_cargo_level_3,scan_cargo_level_7,scan_cargo_level_14,scan_cargo_level_30,scan_cargo_level_avg,click_cargo_level_3,click_cargo_level_7,click_cargo_level_14,click_cargo_level_30,click_cargo_level_avg,call_cargo_level_3,call_cargo_level_7,call_cargo_level_14,call_cargo_level_30,call_cargo_level_avg
0,15090280313811400,390939613,1,False,-1.0,0.0,1.0,0.0,0.0,False,True,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
1,15090280313811400,390994195,1,False,0.0,0.0,3.0,0.0,0.0,False,False,True,False,False,True,True,False,False,False,False,True,True,True,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
2,15090280313811400,391142389,1,False,-1.0,0.0,1.0,0.0,0.0,False,True,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
3,15090280313811400,391165453,1,False,1.0,1.0,7.0,0.0,1.0,False,False,False,False,False,True,True,False,False,False,False,False,False,False,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3
4,15090280313811400,391193650,1,False,-1.0,0.0,1.0,0.0,0.0,False,False,False,False,False,True,True,False,False,False,False,True,True,True,0,1,1,1,168,0,2,2,2,16,1,2,2,2,3


In [4]:
X.keys()

Index(['user_id', 'id', 'gender', 'install_city_match', 'cargo_type', 'handling_type', 'truck_count', 'highway', 'lcl_cargo', 'truck_type_cargo_match_first', 'truck_type_cargo_match_second', 'truck_type_cargo_match_third', 'truck_type_cargo_match_fourth', 'truck_type_driver_match_first', 'truck_len_match', 'regular_subscribe_match', 'regular_search_3_match', 'regular_search_7_match', 'regular_search_14_match', 'regular_search_30_match', 'line_match', 'line_30days_match', 'line_60days_match', 'scan_cargo_level_3', 'scan_cargo_level_7', 'scan_cargo_level_14', 'scan_cargo_level_30', 'scan_cargo_level_avg', 'click_cargo_level_3', 'click_cargo_level_7', 'click_cargo_level_14', 'click_cargo_level_30', 'click_cargo_level_avg', 'call_cargo_level_3', 'call_cargo_level_7', 'call_cargo_level_14', 'call_cargo_level_30', 'call_cargo_level_avg'], dtype='object')

In [6]:
keylist = ['user_id', 'id','truck_count','scan_cargo_level_avg']
# keylist = ['driver_user_id','cargo_id','cargo_freight_unit', 'cargo_type', 'cargo_handling_type', 'cargo_truck_count', 'cargo_highway', 'cargo_lcl_cargo','cargo_expect_freight', 'cargo_freight_per_mile', 'driver_freight_avg', 'driver_freight_avg_3', 'driver_freight_avg_7', 'driver_freight_avg_14','driver_scan_cargo_level_3', 'driver_scan_cargo_level_7', 'driver_scan_cargo_level_14', 'driver_scan_cargo_level_avg', 'driver_click_cargo_level_3', 'driver_click_cargo_level_7', 'driver_click_cargo_level_14', 'driver_click_cargo_level_avg', 'driver_call_cargo_level_3', 'driver_call_cargo_level_7', 'driver_call_cargo_level_14', 'driver_call_cargo_level_avg', 'shipper_gender', 'shipper_age', 'shipper_is_members', 'shipper_deal_cnt', 'shipper_complaint_cnt_all', 'shipper_click_cargo_cnt_3', 'shipper_click_cnt_3', 'shipper_click_cargo_cnt_7', 'shipper_click_cnt_7', 'shipper_click_cargo_cnt_14', 'shipper_click_cnt_14']
for key in keylist:
    mean = np.mean(X[key])
    max = np.max(X[key])
    min = np.min(X[key])
    if (max > min):
        X[key] = (X[key] - min) / (max - min)

In [13]:
from keras.layers.advanced_activations import LeakyReLU
from keras import regularizers
def my_DNN(x):
  
    x = Dense(200, activation = "linear", name = "dense1",kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization(axis=-1, name = "BN1")(x)
    x = LeakyReLU(alpha=0.1, name = "relu1")(x)
    x = Dropout(0.5, name = "Dout1")(x)
    x = Dense(200,activation = "linear", name = "dense2",kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization(axis=-1, name = "BN2")(x)
    x = LeakyReLU(alpha=0.1,name = "relu2")(x)
    x = Dropout(0.5, name = "Dout2")(x)
    x = Dense(200, activation = "linear", name = "dense3",kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization(axis=-1, name = "BN3")(x)
    x = LeakyReLU(alpha=0.1,name = "relu3")(x)
    x = Dropout(0.5, name = "Dout3")(x)
    x = Dense(200, activation = "linear", name = "dense4",kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization(axis=-1, name = "BN4")(x)
    x = LeakyReLU(alpha=0.1,name = "relu4")(x)
    x = Dense(200, activation = "linear", name = "dense5",kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization(axis=-1, name = "BN5")(x)
    x = LeakyReLU(alpha=0.1,name = "relu5")(x)
    output = Dense(2, activation = "sigmoid", name = "output")(x)
    return output
def my_DNN2(x):
    x = Dense(200, activation = "relu", name = "relu1")(x)
    x = Dense(200, activation = "relu", name = "relu2")(x)
    x = Dense(200, activation = "relu", name = "relu3")(x)
    x = Dense(200, activation = "relu", name = "relu4")(x)
    x = Dense(200, activation = "relu", name = "relu5")(x)
    x = Dense(200, activation = "relu", name = "relu6")(x)
    output = Dense(2, activation= "sigmoid", name = "output")(x)
    return output

In [14]:
import tensorflow as tf
from keras import backend as K
def weighted_RMSE(weights):
    def loss(y_true, y_pred):

#         y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
     
#         y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        return (K.sqrt(K.mean(K.square((y_true - y_pred) * weights))))
    return loss

In [15]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
import tensorflow as tf

In [44]:
y = train_df.label

In [43]:
from keras import backend as K

input = Input(shape = (X.shape[1],), name = "input")
#output = my_DNN(input)
output = my_DNN2(input)
weights = K.constant([1.0, 1.0])

if os.path.exists(MODEL_PATH + "/my_DNN.h5"):
    model = load_model(MODEL_PATH + "/my_DNN.h5",custom_objects={'loss':  weighted_RMSE(weights)})
    print("yes")
else:
    model = Model(input, output)
model =Model(input, output)
optimizer = Adam(lr=0.001)

model.compile(optimizer=optimizer, loss=weighted_RMSE(weights), metrics=['accuracy', "categorical_crossentropy"])
tensorboard = TensorBoard(log_dir='tensorboard')
ckpt = ModelCheckpoint(filepath='checkpoints/ckpt1.hdf5', monitor='val_loss', save_best_only=True)

yes


In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle = True)
print(y_train.tolist().count(0))

1070491


In [46]:
from imblearn.over_sampling import SMOTE
num_0 = y_train.tolist().count(0)
over_sample = SMOTE(ratio={0:num_0, 1:int(num_0*0.7)},random_state = 1234)
over_sample_X, over_sample_Y = over_sample.fit_sample(X_train, y_train)

In [47]:
print(y_train.value_counts()/len(y_train))
print(pd.Series(over_sample_Y).value_counts()/len(over_sample_Y))

0    0.937214
1    0.062786
Name: label, dtype: float64
0    0.588236
1    0.411764
dtype: float64


In [48]:
over_sample_Y= to_categorical(over_sample_Y,num_classes=2)
y_test = to_categorical(y_test,num_classes = 2)

In [41]:
# np.save("train_X.npy", np.array(over_sample_X))
# np.save("train_Y.npy", np.array(over_sample_Y))
# np.save("val_X.npy", np.array(X_test))
# np.save("val_Y.npy", np.array(y_test))

In [ ]:
model.fit(over_sample_X,over_sample_Y,batch_size=128,epochs=7,callbacks=[tensorboard, ckpt],validation_data=(X_test, y_test))
model.save('model/my_DNN.h5')
model.summary()

Train on 1819834 samples, validate on 126912 samples
Epoch 1/7
 174464/1819834 [=>............................] - ETA: 6:41 - loss: 0.4648 - acc: 0.6514 - categorical_crossentropy: 0.6234

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis = 1)
y_test = np.argmax(y_test, axis = 1)
print(y_pred.shape)
print(y_test.shape)
print(y_pred.tolist().count(1))

In [ ]:
import math

def calc_score(y_test, y_pred):
    assert(len(y_test) == len(y_pred))
    error = 0
    score_weight = {
        0: 1,
        1: 2,
        2: 2.5
    }
    for y_t, y_p in zip(y_test, y_pred):
        error += (y_t * score_weight[y_t] - y_p) ** 2
    RMSE = math.sqrt(error / len(y_test))
    print('RMSE: ', RMSE)
    print('Score: ', 1 / (1 + RMSE))


In [ ]:
calc_score(y_test, y_pred)

In [ ]:
y1 = np.zeros((63456,))
calc_score(y_test,y1)